In [ ]:
from classification_classes import * 

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping
import wandb
import argparse
import re
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

MODEL_NAME_OR_PATH = "bert-base-german-cased"
DATASETS = ["RP-Crowd-3", "RP-Crowd-2", "RP-Mod"]
DATASET_PATHS = ["./Datasets/RP-Crowd-3-folds.csv", "./Datasets/RP-Crowd-2-folds.csv", "./Datasets/RP-Mod-folds.csv", \
    "./Datasets/resampled/200shap-folds.csv"]
# DATASET_PATH = "/home/dobby/Datasets/resampled/200shap-folds.csv"
# WANDB_PROJECT_NAME = f"{MODEL_NAME_OR_PATH}-all-datasets"
OUTPUT_DIR = f"./german-bert-results/"

TUNING_LEARNING_RATE = False
model_name = "german-bert"
model_class = BertFineTuner

CONFIGS = [
    {
        "model_class": Enc1T5,
        "learning_rate": 0.0001,
        "weight_decay": 0.1, 
        "model_name_or_path": "GermanT5/t5-efficient-oscar-german-small-el32"
    }, 
    {
        "model_class": BertFineTuner,
        "learning_rate": 0.0001, 
        "weight_decay": 0.1, 
        "model_name_or_path": "bert-base-german-cased"
    }, 
    {
        "model_class": T5FineTuner, 
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "model_name_or_path": "GermanT5/t5-efficient-oscar-german-small-el32"
    }, 
    {
        "model_class": T5FineTuner, 
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "model_name_or_path": "google/mt5-small"
    },
    {
        "model_class": T5FineTuner, 
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "model_name_or_path": "google/mt5-base"
    },
    {
        "model_class": T5FineTuner, 
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "model_name_or_path": "GermanT5/german-t5-oscar-ep1-prompted-germanquad"
    },
    {
        "model_class": BertFineTuner,
        #TODO: change these
        "learning_rate": 0.0001, 
        "weight_decay": 0.1,
        "model_name_or_path": "xlm-roberta-base"
    }
]

for config in CONFIGS:
    # pick output_dir
    # set args to the right values
    # initialize wandb project names
    # and wandb logger & checkpoint logger & early stop callback
    # set summary metrics

    # for i in range(num_times):
    # train model
    # keep track of max accuracy, f1, recall, precision
    # figure out how to get the run time of the model



#TODO: set up the configs for running on different datasets with different classes
#TODO: how do we get the summaries for each
#TODO: save checkpoints of the models as well in a folder?
#TODO: somehow keep track of the time that each model took to train?
for source in DATASET_PATHS:
        train_inputs, train_targets, val_inputs, val_targets = get_folds_classification(source)

        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

        train_dataset = RPClassificationDataset(tokenizer, train_inputs, train_targets)
        valid_dataset = RPClassificationDataset(tokenizer, val_inputs, val_targets)
        args_dict = dict(
                model_name_or_path=MODEL_NAME_OR_PATH,
                gradient_accumulation_steps=16,
                weight_decay=0.1,
                learning_rate=1e-5,
                adam_epsilon=1e-8,
                adam_betas=(0.9,0.999),
                num_train_epochs=30,
                n_gpu=1,
                train_batch_size=8,
                eval_batch_size=8,
                data_dir="", # path for data files
                output_dir=OUTPUT_DIR, # path to save the checkpoints
                dataset_name=dataset,
                max_seq_length=512,
                early_stop_callback=True,
                fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
                opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
                max_grad_norm=0.5, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
                seed=42,
                train_dataset=train_dataset,
                val_dataset=valid_dataset,
                warmup_steps=0
                )
        args = argparse.Namespace(**args_dict)
        args.auto_lr_find = "learning_rate"

        train_params = dict(
                accumulate_grad_batches=args.gradient_accumulation_steps,
                auto_lr_find=True,
                gpus=args.n_gpu,
                max_epochs=args.num_train_epochs,
                precision= 16 if args.fp_16 else 32,
                amp_level=args.opt_level,
                gradient_clip_val=args.max_grad_norm,
                # enable_checkpointing=checkpoint_callback,
                callbacks=[],
                # callbacks=[EarlyStopping(monitor="val/accuracy", patience=5, mode="max")],
                # callbacks=[raytuner_callback],
                # callbacks=[LoggingCallback()],
                amp_backend="apex"
                )
        wandb_project_name = f"{dataset}-hyperparameter-search-{model_name}"
        possible_weight_decays = [0.1]
        early_stop_callback = EarlyStopping(monitor="val_accuracy", patience=3, mode="max")
        for wd in possible_weight_decays:
                if TUNING_LEARNING_RATE:
                        
                        args.weight_decay = wd
                        model = BertFineTuner(args)
                        init_trainer = pl.Trainer(**train_params)
                        print("*" * 100)
                        print(f"{dataset} Learning Rate Tuning")
                        lr_finder = init_trainer.tuner.lr_find(model)
                        # print(lr_finder.results)
                        fig = lr_finder.plot(suggest=True)
                        fig.show()
                        new_lr = lr_finder.suggestion()
                        print(f"Best Learning Rate is: {new_lr}")

                        # update with the best learning rate
                        possible_learning_rates = [1e-4, new_lr, 1e-5]
                        # possible_learning_rates = np.power(10, rand.uniform(-6, np.log10(new_lr) + 1, 3))
                else:
                        possible_learning_rates = [1e-4]
                
                for lr in possible_learning_rates:
                        config = {
                                "learning_rate": lr,
                                "weight_decay": wd, 
                                "num_train_epochs": 20
                        }

                        run_name = ""
                        for key in config.keys():
                                run_name += f"-{key}-{config[key]}"
                                args_dict[key] = config[key]
                        args = argparse.Namespace(**args_dict)

                        # get train params and update with wandb logger, checkpoint callback, and early stopping callback
                        early_stop_callback = EarlyStopping(monitor="val_accuracy", patience=1, mode="max")
                        wandb.finish()
                                                
                        wandb_logger = WandbLogger(project=wandb_project_name, 
                                name=run_name)
                        wandb.define_metric("val_accuracy", summary="max")

                        checkpoint_callback = pl.callbacks.ModelCheckpoint(
                                                        dirpath=args.output_dir + "/" + run_name, 
                                                        filename="{epoch}-{val_accuracy:.2f}-{val_loss:.2f}", 
                                                        monitor="val_accuracy", mode="max", save_top_k=5
                                                        )
                        
                        train_params["logger"] = wandb_logger
                        train_params["callbacks"] = [early_stop_callback, checkpoint_callback]

                        model = model_class(args)
                        trainer = pl.Trainer(**train_params)
                        trainer.fit(model)